In [2]:
import os
from PIL import Image, ImageFilter, ImageEnhance
import torch
import torchvision.transforms as transforms
import numpy as np
import random
import imgaug as ia
ia.seed(1)
from imgaug import augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import cv2

In [2]:
real_lesion_dir = 'D:/FYP/all_training/real_positive_samples/'
real_no_lesion_dir = 'D:/FYP/all_training/real_negative_samples/'

real_aug_lesion_dir = 'D:/FYP/all_training/aug_real_positive_samples/'
real_aug_no_lesion_dir = "D:/FYP/all_training/aug_real_negative_samples/"

synthetic_lesion_dir = "D:/FYP/all_training/synthetic_positive_samples/"
synthetic_no_lesion_dir = "D:/FYP/all_training/synthetic_negative_samples/"

synthetic_aug_lesion_dir = "D:/FYP/all_training/aug_synthetic_positive_samples/"
synthetic_aug_no_lesion_dir = "D:/FYP/all_training/aug_synthetic_negative_samples/"

bb_real_dir = 'D:/FYP/all_training/yolo_label_real_positive/'
bb_synthetic_dir = "D:/FYP/all_training/yolo_label_synthetic_positive/"

bb_aug_real_dir = "D:/FYP/all_training/yolo_label_aug_real_positive/"
bb_aug_synthetic_dir = "D:/FYP/all_training/yolo_label_aug_synthetic_positive/"

In [3]:
# # Define your transforms
# augmentations = [
#     transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5),
#     transforms.Lambda(lambda x: x.filter(ImageFilter.GaussianBlur(radius=random.uniform(0.1, 2.0)))),
#     transforms.Lambda(lambda x: x.filter(ImageFilter.UnsharpMask(radius=random.uniform(0.5, 1.5), percent=200))),
#     transforms.Lambda(lambda x: ImageEnhance.Brightness(x).enhance(random.uniform(0.5, 1.5))),
#     transforms.Lambda(lambda x: ImageEnhance.Color(x).enhance(random.uniform(0.5, 1.5))),
#     # transforms.Lambda(lambda x: x + torch.randn_like(x) * 0.05)  # add gaussian noise
# ]

augmentations = iaa.Sequential([
    iaa.Fliplr(0.5), # horizontal flips
    iaa.Crop(percent=(0, 0.1)), # random crops
    # Small gaussian blur with random sigma between 0 and 0.5.
    # But we only blur about 50% of all images.
    iaa.Sometimes(
        0.5,
        iaa.GaussianBlur(sigma=(0, 0.5))
    ),
    # Strengthen or weaken the contrast in each image.
    iaa.LinearContrast((0.75, 1.5)),
    # Add gaussian noise.
    # For 50% of all images, we sample the noise once per pixel.
    # For the other 50% of all images, we sample the noise per pixel AND
    # channel. This can change the color (not only brightness) of the
    # pixels.
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255)),
    # Make some images brighter and some darker.
    # In 20% of all cases, we sample the multiplier once per channel,
    # which can end up changing the color of the images.
    iaa.Multiply((0.8, 1.2)),
    # Apply affine transformations to each image.
    # Scale/zoom them, translate/move them, rotate them and shear them.
    iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        shear=(-8, 8)
    ),
    iaa.ScaleX((1.0, 1.5)),
], random_order=True) # apply augmenters in random order

In [4]:
sometimes = lambda aug: iaa.Sometimes(0.5, aug)

augmenters = iaa.Sequential(
    [
        #
        # Apply the following augmenters to most images.
        #
        iaa.Fliplr(0.5), # horizontally flip 50% of all images

        # crop some of the images by 0-10% of their height/width
        sometimes(iaa.Crop(percent=(0, 0.1))),

        # Apply affine transformations to some of the images
        # - scale to 80-120% of image height/width (each axis independently)
        # - translate by -20 to +20 relative to height/width (per axis)
        # - rotate by -45 to +45 degrees
        # - shear by -16 to +16 degrees
        # - order: use nearest neighbour or bilinear interpolation (fast)
        # - mode: use any available mode to fill newly created pixels
        #         see API or scikit-image for which modes are available
        # - cval: if the mode is constant, then use a random brightness
        #         for the newly created pixels (e.g. sometimes black,
        #         sometimes white)
        sometimes(iaa.Affine(
            scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
            translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
            rotate=(-45, 45),
            shear=(-16, 16),
            order=[0, 1],
            cval=(0, 255),
            mode=ia.ALL
        )),

        #
        # Execute 0 to 5 of the following (less important) augmenters per
        # image. Don't execute all of them, as that would often be way too
        # strong.
        #
        iaa.SomeOf((2, 5),
            [
                # Convert some images into their superpixel representation,
                # sample between 20 and 200 superpixels per image, but do
                # not replace all superpixels with their average, only
                # some of them (p_replace).
                sometimes(
                    iaa.Superpixels(
                        p_replace=(0, 1.0),
                        n_segments=(20, 200)
                    )
                ),

                # Blur each image with varying strength using
                # gaussian blur (sigma between 0 and 3.0),
                # average/uniform blur (kernel size between 2x2 and 7x7)
                # median blur (kernel size between 3x3 and 11x11).
                iaa.OneOf([
                    iaa.GaussianBlur((0, 3.0)),
                    iaa.AverageBlur(k=(2, 7)),
                    iaa.MedianBlur(k=(3, 11)),
                ]),

                # Sharpen each image, overlay the result with the original
                # image using an alpha between 0 (no sharpening) and 1
                # (full sharpening effect).
                iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)),

                # Same as sharpen, but for an embossing effect.
                iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)),

                # Search in some images either for all edges or for
                # directed edges. These edges are then marked in a black
                # and white image and overlayed with the original image
                # using an alpha of 0 to 0.7.
                sometimes(iaa.OneOf([
                    iaa.EdgeDetect(alpha=(0, 0.7)),
                    iaa.DirectedEdgeDetect(
                        alpha=(0, 0.7), direction=(0.0, 1.0)
                    ),
                ])),

                # Add gaussian noise to some images.
                # In 50% of these cases, the noise is randomly sampled per
                # channel and pixel.
                # In the other 50% of all cases it is sampled once per
                # pixel (i.e. brightness change).
                iaa.AdditiveGaussianNoise(
                    loc=0, scale=(0.0, 0.05*255), per_channel=0.5
                ),

                # Either drop randomly 1 to 10% of all pixels (i.e. set
                # them to black) or drop them on an image with 2-5% percent
                # of the original size, leading to large dropped
                # rectangles.
                iaa.OneOf([
                    iaa.Dropout((0.01, 0.1), per_channel=0.5),
                    iaa.CoarseDropout(
                        (0.03, 0.15), size_percent=(0.02, 0.05),
                        per_channel=0.2
                    ),
                ]),

                # Invert each image's channel with 5% probability.
                # This sets each pixel value v to 255-v.
                iaa.Invert(0.05, per_channel=True), # invert color channels

                # Add a value of -10 to 10 to each pixel.
                iaa.Add((-10, 10), per_channel=0.5),

                # Change brightness of images (50-150% of original value).
                iaa.Multiply((0.5, 1.5), per_channel=0.5),

                # Improve or worsen the contrast of images.
                iaa.LinearContrast((0.5, 2.0), per_channel=0.5),

                # Convert each image to grayscale and then overlay the
                # result with the original with random alpha. I.e. remove
                # colors with varying strengths.
                iaa.Grayscale(alpha=(0.0, 1.0)),

                # In some images move pixels locally around (with random
                # strengths).
                sometimes(
                    iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)
                ),

                # In some images distort local areas with varying strength.
                sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.05)))
            ],
            # do all of the above augmentations in random order
            random_order=True
        )
    ],
    # do all of the above augmentations in random order
    random_order=True
)

In [4]:
import imageio

In [5]:
import cv2

In [15]:
with Image.open("D:/FYP/all_training/real_negative_samples/ash-common-6.jpg") as img:
	img.show()

In [6]:
def read_yolo_format(file_path, img_width, img_height):
    """
    Function to convert bounding box coordinates from YOLO format
    (class_id, x_center, y_center, width, height)
    """
    bounding_boxes = []
    with open(file_path, 'r') as f:
        for line in f:
            class_id, x_center, y_center, width, height = map(float, line.strip().split())
            x1 = (x_center - width / 2) * img_width
            y1 = (y_center - height / 2) * img_height
            x2 = x1 + width * img_width
            y2 = y1 + height * img_height

            bounding_boxes.append(BoundingBox(x1=x1, y1=y1, x2=x2, y2=y2))

    return BoundingBoxesOnImage(bounding_boxes, shape=(img_height, img_width, 3))

def convert_to_yolo_label(x1,y1,x2,y2, img_width, img_height):
    """
    Function to convert bounding box coordinates to YOLO format
    """
    x_center = (x1+x2)/2
    y_center = (y1+y2)/2
    width = x2-x1
    height = y2-y1

    # Normalize to be between 0 and 1
    x_center /= img_width
    y_center /= img_height
    width /= img_width
    height /= img_height

    return x_center, y_center, width, height

In [7]:
import matplotlib.pyplot as plt
def augment_and_save_images(augmentations, input_dir, output_dir, bb_input_dir=None, bb_output_dir=None):
    # Loop over all files in the input directory
    for filename in os.listdir(input_dir):

        img = cv2.imread(os.path.join(input_dir, filename))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if bb_input_dir is not None: 
            yolo_file_path = os.path.splitext(filename)[0] + '.txt'
            h, w = img.shape[:2]

            bbs = read_yolo_format(os.path.join(bb_input_dir,yolo_file_path), w, h)

            img, bbs_aug = augmentations(image=img, bounding_boxes=bbs)
            bbs_aug = bbs_aug.remove_out_of_image().clip_out_of_image()

            x1 = bbs_aug[0][0][0]
            y1 = bbs_aug[0][0][1]
            x2 = bbs_aug[0][1][0]
            y2 = bbs_aug[0][1][1]

            x_center, y_center, width, height = convert_to_yolo_label(x1,y1,x2,y2,w,h)
            with open(os.path.join(bb_output_dir, 'augmented_' + yolo_file_path), 'w') as f:
                class_id = 0 
                f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")
            
        else:
            img_aug = augmentations(image=img)

        imageio.imwrite(os.path.join(output_dir, 'augmented_' + filename), img)


In [74]:
augment_and_save_images(augmentations, real_lesion_dir, real_aug_lesion_dir, bb_input_dir=bb_real_dir, bb_output_dir=bb_aug_real_dir)

In [8]:
augment_and_save_images(augmentations, synthetic_lesion_dir, synthetic_aug_lesion_dir, bb_input_dir=bb_synthetic_dir, bb_output_dir=bb_aug_synthetic_dir)

In [1]:
unrendered_dir = "D:/FYP/unrendered"

In [9]:
augment_and_save_images(unrendered_dir, aug_no_lesion_dir, augmentations)

In [27]:
# Apply augmentations and save augmented images
# augment_and_save_images(lesion_dir, aug_lesion_dir, augmentations)
augment_and_save_images(no_lesion_dir, aug_no_lesion_dir, augmentations)

In [4]:
image_path = "D:/FYP/yolo_test_data/images/"
import os
for filename in os.listdir(image_path):
 # create a new filename by replacing the image extension with .txt
	new_filename = os.path.splitext(filename)[0] + '.txt'
	new_filepath = os.path.join(image_path, new_filename)
		
	# create an empty text file if it doesn't exist
	if not os.path.exists(new_filepath):
		open(new_filepath, 'a').close()